<a href="https://colab.research.google.com/github/M-Amrollahi/RecSys-movieLens/blob/main/recom_sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download movielens dataset

In [63]:
!wget -nv https://files.grouplens.org/datasets/movielens/ml-100k.zip
!wget -nv https://files.grouplens.org/datasets/movielens/ml-100k-README.txt

!unzip -qo ml-100k.zip

2023-04-04 09:18:53 URL:https://files.grouplens.org/datasets/movielens/ml-100k.zip [4924029/4924029] -> "ml-100k.zip.4" [1]
2023-04-04 09:18:53 URL:https://files.grouplens.org/datasets/movielens/ml-100k-README.txt [6748/6748] -> "ml-100k-README.txt.4" [1]


In [ ]:

!cat ./ml-100k/README

## Install Faiss from conda

In [69]:
!wget -nv https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.9/dist-packages/
!pip install -q mkl

2023-04-04 09:24:49 URL:https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9c5c376961204909d87/5aa7f0a65571b411e5c259be?response-content-disposition=attachment%3B%20filename%3D%22faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=600&X-Amz-Date=20230404T092448Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEA4aCXVzLWVhc3QtMSJHMEUCIAKMVsojGb%2FXI3MxOGMOg86UFFmUYEjGAlGw9gfwxgxIAiEAyf%2Bjz7SrG10KpIix6Mx23g%2FtjwJuIzTEoXDp5W7MMrUqvAUI5%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAAGgw0NTU4NjQwOTgzNzgiDJezzI0GDxM85yfASSqQBaAqFzC%2FKf2qnqzz0Bvf0zNp%2FOLpnMi0Vqozmxodo0T1%2F95ZxngnCFjxF4arMiKkUf1oPQHGIIQ%2Bm2LDvAbrM%2Fr3RPZjGcM%2FZ3jqGsWVbRAwa8%2BqAcObpZLRh%2ByXrcT4EynIQJYqPpqvcoolgsyaDDjWshPuEeQVlEWCwXQlMk9xAFjoYmssFaqQJcpJu8OMg5Ak8xldbghDUGGAalhHSq8GcWsobp0cIzt3yezIR0EB3rNbvOMybdUNK8RIj25%2Bo9%2Bi2RC2ZFejK4C2U8x8tvXSzk2c90bsYU

In [70]:
!pip install -qU sentence-transformers

## import packages

In [71]:
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import top_k_accuracy_score
import numpy as np
from pprint import pprint
from tqdm.notebook import tqdm

from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F

from sentence_transformers import SentenceTransformer

import faiss

import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 100

## Read datasets and prepare to join dataframes 

In [7]:
## data
df1 = pd.read_csv("./ml-100k/u.data", 
                  delimiter="\t",
                  header=None,
                  names=["user_id" , "item_id" , "rating" , "timestamp"])
df1.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [8]:
## Genre
#!head ./ml-100k/u.genre

df5 = pd.read_csv("./ml-100k/u.genre",
                  delimiter="|",                  
                  header=None,
                  names=["genre","genre_id"]
)
df5.head()

,genre,genre_id
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4


In [9]:
## Movie
#!head ./ml-100k/u.item

df3 = pd.read_csv("./ml-100k/u.item",
                  delimiter="|",
                  encoding="cp1252",
                  header=None,
                  names=["item_id", "movie_title", "release_date","video_release_date", "imdb_url","unknown", "Action" , "Adventure",  "Animation", "Children" , "Comedy" , "Crime" , "Documentary" , "Drama", "Fantasy", "Film_Noir" , "Horror"  ,"Musical" ,"Mystery", "Romance" , "Sci_Fi", "Thriller" , "War" , "Western" ]
)
df3.head()

,item_id,movie_title,release_date,video_release_date,imdb_url,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [10]:
## user
#!head ./ml-100k/u.user

df2 = pd.read_csv("./ml-100k/u.user",
                  delimiter="|",
                  header=None,
                  names=["user_id","age", "gender","occupation", "zip_code"])
df2.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [11]:
## occupation
#!cat ./ml-100k/u.occupation

In [12]:
df10 = pd.merge( pd.merge(df1,df2, on="user_id", how="inner"), df3 , on="item_id", how="inner")
df10.head()

,user_id,item_id,rating,timestamp,age,gender,occupation,zip_code,movie_title,release_date,video_release_date,imdb_url,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,196,242,3,881250949,49,M,writer,55105,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,305,242,5,886307828,23,M,programmer,94086,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6,242,4,883268170,42,M,executive,98101,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,234,242,4,891033261,60,M,retired,94702,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,63,242,3,875747190,31,M,marketing,75240,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


## Encode the gender column to one-hot

In [13]:
df11 = pd.get_dummies(df10,prefix=["gen"],columns=["gender"])
df11.head()

,user_id,item_id,rating,timestamp,age,occupation,zip_code,movie_title,release_date,video_release_date,imdb_url,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western,gen_F,gen_M
0,196,242,3,881250949,49,writer,55105,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,305,242,5,886307828,23,programmer,94086,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,6,242,4,883268170,42,executive,98101,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,234,242,4,891033261,60,retired,94702,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,63,242,3,875747190,31,marketing,75240,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## normalize the age and timestamp

In [14]:
model_norm = StandardScaler()
df11[["age","timestamp"]] = model_norm.fit_transform(df11[["age", "timestamp"]])

In [15]:
model_labelNorm = LabelEncoder()
df11["occupation"] = model_labelNorm.fit_transform(df11["occupation"])

## smooth cleaning on movie title

In [16]:
df11["movie_title"] = df11["movie_title"].str.lower().str.replace(r"[\:\&\,]","")

<ipython-input-16-81fe8036bc2a>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df11["movie_title"] = df11["movie_title"].str.lower().str.replace(r"[\:\&\,]","")


In [17]:
df11["user_id"] = df11["user_id"] - 1
df11["item_id"] = df11["item_id"] - 1

## Preparing negative samples
For simplicity, we consider each record in dataset as positive and each potential record as negative.\
Then we create the y as target and name it "like" and label it as 0 or 1.

In [18]:
## Negative samples
ngSize = 80
df_neg = df11.loc[0:2]
for j in df11["user_id"].unique():
    dft = df11.loc[ df11["user_id"] != j].sample(ngSize, random_state=j+1)
    dft["user_id"] = j
    df_neg = pd.concat((df_neg,dft), axis=0)

In [19]:
df_neg = df_neg.iloc[2:]
df_neg["like"] = 0

df11["like"] = 1

df_train, df_test = train_test_split(df11, train_size=.8, random_state=42, shuffle=True)


df_train = pd.concat((df_train, df_neg), axis=0)

<ipython-input-19-ef4152031451>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg["like"] = 0


In [20]:
df_test = df_test.reset_index(drop=True )
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)


df_train.sample(3)

,user_id,item_id,rating,timestamp,age,occupation,zip_code,movie_title,release_date,video_release_date,imdb_url,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western,gen_F,gen_M,like
47634,847,198,5,0.657485,1.126926,4,02146,bridge on the river kwai the (1957),01-Jan-1957,NaN,http://us.imdb.com/M/title-exact?Bridge%20on%2...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1
55068,294,153,5,-0.750595,-0.170364,3,50325,monty python's life of brian (1979),01-Jan-1979,NaN,http://us.imdb.com/M/title-exact?Life%20of%20B...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
148150,600,505,4,-0.749266,0.435038,0,48103,rebel without a cause (1955),01-Jan-1955,NaN,http://us.imdb.com/M/title-exact?Rebel%20Witho...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0


In [21]:
df_train.rating.value_counts()


4    53324
3    42079
5    32948
2    17720
1     9370
Name: rating, dtype: int64

## Encode the title of the movie with SentenceTransformer and save then in a dict to be used next

In [22]:
model_strans = SentenceTransformer('all-MiniLM-L6-v2')
dict_titlEmb = {}
for i in df11["movie_title"].unique():
    dict_titlEmb[i] = torch.tensor(model_strans.encode(i))


# Deep model in PyTorch

In [23]:
class cls_dataset(Dataset):
    """
    return as user, item, like, rating
    """
    def __init__(self, data, ):
        super().__init__()

        
        self.m_data = data

    def __len__(self):
        return len(df11)

    def __getitem__(self, index) :
        ts_title = dict_titlEmb[self.m_data.loc[index,"movie_title"]]

        return torch.tensor(self.m_data.loc[index,["user_id", "age","occupation","gen_F","gen_M"]] , dtype=torch.float32),\
         torch.cat(( torch.tensor(self.m_data.loc[index, ["item_id", "timestamp","unknown",'Action', 'Adventure', 'Animation', 'Children','Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film_Noir','Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War','Western']], dtype=torch.float32), ts_title), dim=0),\
         torch.tensor([self.m_data.loc[index,"like"]]),\
         torch.tensor([self.m_data.loc[index,"rating"]])

## Build a two-tower model

In [24]:
class cls_model(nn.Module):
    def __init__(self, userCount, itemCount, user_embSize = 16, item_embSize=16):
        super().__init__()

        self.m_userEmb = nn.Embedding(userCount , user_embSize)
        self.m_itemEmb = nn.Embedding(itemCount, item_embSize)

        self.m_modelUser = nn.Sequential(
            nn.Linear(20, 16),
            nn.ReLU(),
            nn.Linear(16,16))
        self.m_modelItem = nn.Sequential(
            nn.Linear(420, 128),
            nn.ReLU(),
            nn.Linear(128,16)
            )
        self.m_modelClassify = nn.Sequential(
            nn.Linear(32,32), 
            nn.ReLU(), 
            nn.Linear(32,2)
            )

    def forward(self, dataUser, dataItem):
        embu = self.m_userEmb.forward(dataUser[:,0].long())
        embi = self.m_itemEmb.forward(dataItem[:,0].long())

        
        inputU = torch.cat((dataUser[:,1:], embu), dim=1)
        inputI = torch.cat((dataItem[:,1:], embi), dim=1)

        logitsU = self.m_modelUser.forward(inputU)
        logitsI = self.m_modelItem.forward(inputI)

        logits = self.m_modelClassify.forward( torch.cat((logitsU, logitsI), dim=1) )
        return logits

    def predict(self, userID: torch.tensor):
        """
        Get the similarity between userIDs and all available items
        """
        embu = self.m_userEmb.forward(userID.long())
        embi = self.m_itemEmb.weight.data

        res = embu @ embi.T
        normU = torch.linalg.norm(embu, dim=1, ord=2)
        normI = torch.linalg.norm(embi, dim=1, ord=2)

        normU.unsqueeze_(dim=1)
        normI.unsqueeze_(dim=0)

        res = res / (normU @ normI)

        return res

## Training and evaluating

In [25]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device is {device}")

epochs = 3
ds = cls_dataset(df_train)
ds_trainLoader = DataLoader(ds, batch_size=1500)
ds_testLoader = DataLoader(torch.tensor(df_test[["user_id", "item_id"]].to_numpy()), batch_size=1000)

modelRec = cls_model(len(df11["user_id"].unique()) , len(df11["item_id"].unique()))
modelRec = modelRec.to(device)

criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(modelRec.parameters(), lr=5e-3)

modelRec.train()

for j in range(epochs):
    loss_acc = 0
    for i, (x1,x2,y,_) in enumerate(tqdm(ds_trainLoader)):
        x1,x2,y = x1.to(device), x2.to(device), y.to(device)

        logits = modelRec.forward(x1,x2)

        loss = criterion.forward(logits, y.squeeze())

        optim.zero_grad()

        loss.backward()

        optim.step()

        loss_acc += loss.item()
        if i%10 == 0:
            print(f"Train ce loss {loss_acc/10}")
            loss_acc = 0

            modelRec.eval()

            
            total_acc=0.0
            for k, batch in enumerate(ds_testLoader):
                batch = batch.to(device)

                pred = modelRec.predict(batch[:,0])

                acc = top_k_accuracy_score(batch[:,1].detach().cpu().numpy(), pred.detach().cpu().numpy(), k=10, labels= range(pred.shape[1]))
                
                #print(acc)
                total_acc += acc
            print(f"Total val top-k-accuracy: {total_acc/(k+1)}")
            modelRec.train()
    
    

Device is cuda


  0%|          | 0/67 [00:00<?, ?it/s]

Train ce loss 0.06959684491157532
Total val top-k-accuracy: 0.005900000000000001
Train ce loss 0.6934185266494751
Total val top-k-accuracy: 0.00595
Train ce loss 0.6915694415569306
Total val top-k-accuracy: 0.0058000000000000005
Train ce loss 0.6853868842124939
Total val top-k-accuracy: 0.005800000000000001
Train ce loss 0.6740423977375031
Total val top-k-accuracy: 0.006
Train ce loss 0.6521379828453064
Total val top-k-accuracy: 0.00595
Train ce loss 0.6288148522377014
Total val top-k-accuracy: 0.00595


  0%|          | 0/67 [00:00<?, ?it/s]

Train ce loss 0.058207809925079346
Total val top-k-accuracy: 0.0061
Train ce loss 0.5794522404670716
Total val top-k-accuracy: 0.006050000000000001
Train ce loss 0.5533526301383972
Total val top-k-accuracy: 0.005850000000000001
Train ce loss 0.5262650281190873
Total val top-k-accuracy: 0.005750000000000001
Train ce loss 0.5030564367771149
Total val top-k-accuracy: 0.00585
Train ce loss 0.4753230005502701
Total val top-k-accuracy: 0.005950000000000001
Train ce loss 0.45198115408420564
Total val top-k-accuracy: 0.005900000000000001


  0%|          | 0/67 [00:00<?, ?it/s]

Train ce loss 0.04299101531505585
Total val top-k-accuracy: 0.0058000000000000005
Train ce loss 0.403478792309761
Total val top-k-accuracy: 0.005900000000000001
Train ce loss 0.3905055433511734
Total val top-k-accuracy: 0.0058000000000000005
Train ce loss 0.3645411849021912
Total val top-k-accuracy: 0.005800000000000001
Train ce loss 0.33803906142711637
Total val top-k-accuracy: 0.005800000000000001
Train ce loss 0.3200076758861542
Total val top-k-accuracy: 0.005700000000000002
Train ce loss 0.3035527318716049
Total val top-k-accuracy: 0.0056500000000000005


In [26]:
model_pca = PCA(2)
pca_user = model_pca.fit_transform(modelRec.m_userEmb.weight.detach().cpu().numpy())
pca_item = model_pca.fit_transform(modelRec.m_itemEmb.weight.detach().cpu().numpy())

In [ ]:
plt.scatter(pca_user[:,0], pca_user[:,1])

In [ ]:
plt.scatter(pca_item[:,0], pca_item[:,1])

In [52]:
class cls_modelRank(nn.Module):
    def __init__(self, userEmb, itemEmb):
        super().__init__()

        self.m_userEmb = userEmb
        self.m_itemEmb = itemEmb

        self.m_modelClassify = nn.Sequential(
            nn.Linear(32,32), 
            nn.ReLU(), 
            nn.Linear(32,1)
            )

    def forward(self, dataUser, dataItem):
        embu = self.m_userEmb.forward(dataUser.long())
        embi = self.m_itemEmb.forward(dataItem.long())


        logits = self.m_modelClassify.forward( torch.cat((embu, embi), dim=1) )
        return logits

    def f_getUserEmb(self, ids):
        assert type(ids) == torch.Tensor
        return self.m_userEmb.forward(ids)


In [53]:
epochs = 3

ds_trainLoader = DataLoader(torch.tensor(df_train[["user_id", "item_id","rating"]].to_numpy(), dtype=torch.float32), batch_size=1000)
ds_testLoader = DataLoader(torch.tensor(df_test[["user_id", "item_id","rating"]].to_numpy(), dtype=torch.float32), batch_size=1000)

modelRank = cls_modelRank(modelRec.m_userEmb, modelRec.m_itemEmb).to(device)


criterion = nn.MSELoss()
optim = torch.optim.Adam(modelRank.parameters(), lr=5e-3)

modelRank.train()

for j in range(epochs):
    loss_acc = 0
    for i, x in enumerate(tqdm(ds_trainLoader)):
        x = x.to(device)

        logits = modelRank.forward(x[:,0], x[:,1])

        loss = criterion.forward(logits, x[:,2].squeeze())

        optim.zero_grad()

        loss.backward()

        optim.step()

        loss_acc += loss.item()
        if i%10 == 0:
            print(f"Train train MSE loss {loss_acc/10}")
            loss_acc = 0

            modelRank.eval()

            
            total_acc=0.0
            for k, batch in enumerate(ds_testLoader):
                batch = batch.to(device)
                
                y_pred = modelRank.forward(batch[:,0], batch[:,1])

                lossMSE = criterion.forward(batch[:,2], y_pred)
                
                total_acc += lossMSE.item()
            print(f"Total val MSE loss: {total_acc/(k+1)}")
            modelRec.train()
    
    

  0%|          | 0/156 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1000])) that is different to the input size (torch.Size([1000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1000, 1])) that is different to the input size (torch.Size([1000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Train train MSE loss 1.2772955894470215
Total val MSE loss: 12.496755123138428
Train train MSE loss 9.837407445907592
Total val MSE loss: 6.299047493934632
Train train MSE loss 4.217829942703247
Total val MSE loss: 2.557368516921997
Train train MSE loss 2.418681192398071
Total val MSE loss: 2.251816987991333
Train train MSE loss 1.8674386620521546
Total val MSE loss: 1.6698191046714783
Train train MSE loss 1.6031038880348205
Total val MSE loss: 1.5470963060855865
Train train MSE loss 1.4877387404441833
Total val MSE loss: 1.4631243526935578
Train train MSE loss 1.4287452340126037
Total val MSE loss: 1.4241326034069062
Train train MSE loss 1.4162606358528138
Total val MSE loss: 1.3992814481258393
Train train MSE loss 1.380660879611969
Total val MSE loss: 1.3788695931434631
Train train MSE loss 1.3678871750831605
Total val MSE loss: 1.365640652179718
Train train MSE loss 1.3535563230514527
Total val MSE loss: 1.3538377285003662
Train train MSE loss 1.3217895865440368
Total val MSE loss: 

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([441])) that is different to the input size (torch.Size([441, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


  0%|          | 0/156 [00:00<?, ?it/s]

Train train MSE loss 0.12633357048034669
Total val MSE loss: 1.3233357548713685
Train train MSE loss 1.2862695932388306
Total val MSE loss: 1.3215149343013763
Train train MSE loss 1.3128455877304077
Total val MSE loss: 1.3165365159511566
Train train MSE loss 1.2951332449913024
Total val MSE loss: 1.3136372864246368
Train train MSE loss 1.2766772389411927
Total val MSE loss: 1.312382411956787
Train train MSE loss 1.2861409187316895
Total val MSE loss: 1.3094751834869385
Train train MSE loss 1.2823561668395995
Total val MSE loss: 1.3084600150585175
Train train MSE loss 1.2890886425971986
Total val MSE loss: 1.3059955656528472
Train train MSE loss 1.3139785408973694
Total val MSE loss: 1.3050307393074037
Train train MSE loss 1.2907488703727723
Total val MSE loss: 1.3039889216423035
Train train MSE loss 1.2962934732437135
Total val MSE loss: 1.3025465428829193
Train train MSE loss 1.2924437761306762
Total val MSE loss: 1.3023630738258363
Train train MSE loss 1.2739103317260743
Total val MS

  0%|          | 0/156 [00:00<?, ?it/s]

Train train MSE loss 0.12348934412002563
Total val MSE loss: 1.298444539308548
Train train MSE loss 1.2612939953804017
Total val MSE loss: 1.3027167856693267
Train train MSE loss 1.2928236722946167
Total val MSE loss: 1.2987198173999785
Train train MSE loss 1.2762380123138428
Total val MSE loss: 1.296861857175827
Train train MSE loss 1.2596556544303894
Total val MSE loss: 1.2967730224132539
Train train MSE loss 1.2711023092269897
Total val MSE loss: 1.2962116062641145
Train train MSE loss 1.2685937404632568
Total val MSE loss: 1.2969422161579132
Train train MSE loss 1.2774351954460144
Total val MSE loss: 1.2952883064746856
Train train MSE loss 1.3034989476203918
Total val MSE loss: 1.2947211980819702
Train train MSE loss 1.2812050938606263
Total val MSE loss: 1.294508957862854
Train train MSE loss 1.2871447324752807
Total val MSE loss: 1.2942996382713319
Train train MSE loss 1.284367573261261
Total val MSE loss: 1.2942300677299499
Train train MSE loss 1.266446578502655
Total val MSE lo

## Indexing with Faiss

In [55]:
index = faiss.IndexFlatL2(modelRank.m_itemEmb.embedding_dim)

In [56]:
index.add(modelRank.m_itemEmb.weight.data.detach().cpu().numpy())

In [44]:
index.ntotal

1682

## Searching on Faiss

In [61]:
q = modelRank.f_getUserEmb(torch.tensor([1,2,10]).to(device))
q = q.detach().cpu().numpy()

index.search(q,5)

(array([[16.463032, 17.019987, 18.739151, 22.927094, 23.147547],
        [ 8.83136 ,  9.884465, 11.050877, 11.530081, 11.817078],
        [15.861409, 16.309996, 17.235512, 17.304667, 18.139887]],
       dtype=float32),
 array([[1026, 1145, 1650, 1522,  632],
        [ 243,  437,  399, 1120, 1381],
        [1587, 1483, 1182, 1625, 1664]]))